# Intelligent AI ChatBot in Python (NeuralNine)

In [1]:
# Intelligent AI ChatBot in Python
# this is going to dyamically understand and respond to greetings
# but the responses will be static

# our files will be : intents.json

## TRAINING CHATBOT

In [2]:
# imports
import random
import json
import pickle   # for serialization
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### *loading our intents*

In [3]:
# loading file
from google.colab import files
uploaded = files.upload()

Saving intents_hin.json to intents_hin.json


In [6]:
intents =  json.loads(uploaded['intents_hin.json'])
print(intents)
print(intents['उद्देश्य'][3])
print(intents['उद्देश्य'][3]['उपनाम'])

# intents -> उद्देश्य
# tag -> उपनाम
# patterns -> प्रतिरूप

{'उद्देश्य': [{'उपनाम': 'अभिवादन', 'प्रतिरूप': ['नमस्ते', 'अरे', 'शुभ दिवस', 'शुभ प्रभात', 'क्या चल रहा है'], 'प्रतिक्रियाओं': ['नमस्ते', 'मेरे द्वारा आपके लिए क्या किया जा सकता है?']}, {'उपनाम': 'अलविदा', 'प्रतिरूप': ['बाद में मिलते है', 'मैं जा रहा हूँ', 'अलविदा', 'आपका दिन शुभ हो', 'हम बाद में मिलेंगे', 'आप से कल मिलता हूं'], 'प्रतिक्रियाओं': ['अलविदा', 'तुमसे मिलकर खुशी हुई']}, {'उपनाम': 'उम्र', 'प्रतिरूप': ['आपका निर्माता कितने साल का है', 'फ्लोरियन कि उम्र क्या है?', 'तुम्हारी उम्र क्या हैं', 'आपकी उम्र क्या है?', 'उम्र?'], 'प्रतिक्रियाओं': ['मेरा निर्माता २१ साल का है!', '२१ साल!', '२१']}, {'उपनाम': 'नाम', 'प्रतिरूप': ['आपका नाम क्या है', 'मैं आपको क्या बुलाऊं?', 'आपका नाम क्या है', 'आप कौन हैं', 'क्या आप मुझे अपना नाम बता सकते हैं'], 'प्रतिक्रियाओं': ['आप मुझे नियुरल कह सकते हैं!', 'मैं नियुरल हूं', 'मैं नियुरल हूं, फ्लोरियन का सहायक']}, {'उपनाम': 'दुकान', 'प्रतिरूप': ['मैं कुछ खरीदना चाहता हूं', 'आपके उत्पाद क्या हैं?', 'आप क्या सलाह देते हैं?', 'आप क्या बेच रहे हैं?'], 'प्रति

### *Creating intermediate Data*

In [7]:
# creatig useful lists
words = []
classes = []
docs = []

ignore_letters = ["!", ".", ",", "?"]

# iterating over intents
for intent in intents['उद्देश्य']:
  for pat in intent['प्रतिरूप']:
    # tokenizing and adding tokens to the word list
    word_list = nltk.word_tokenize(pat)
    if word_list is not None:
      words.extend(word_list)

      # appending 
      docs.append((word_list, intent['उपनाम']))

      if intent['उपनाम'] not in classes:
        classes.append(intent['उपनाम'])

In [8]:
print(words)

['नमस्ते', 'अरे', 'शुभ', 'दिवस', 'शुभ', 'प्रभात', 'क्या', 'चल', 'रहा', 'है', 'बाद', 'में', 'मिलते', 'है', 'मैं', 'जा', 'रहा', 'हूँ', 'अलविदा', 'आपका', 'दिन', 'शुभ', 'हो', 'हम', 'बाद', 'में', 'मिलेंगे', 'आप', 'से', 'कल', 'मिलता', 'हूं', 'आपका', 'निर्माता', 'कितने', 'साल', 'का', 'है', 'फ्लोरियन', 'कि', 'उम्र', 'क्या', 'है', '?', 'तुम्हारी', 'उम्र', 'क्या', 'हैं', 'आपकी', 'उम्र', 'क्या', 'है', '?', 'उम्र', '?', 'आपका', 'नाम', 'क्या', 'है', 'मैं', 'आपको', 'क्या', 'बुलाऊं', '?', 'आपका', 'नाम', 'क्या', 'है', 'आप', 'कौन', 'हैं', 'क्या', 'आप', 'मुझे', 'अपना', 'नाम', 'बता', 'सकते', 'हैं', 'मैं', 'कुछ', 'खरीदना', 'चाहता', 'हूं', 'आपके', 'उत्पाद', 'क्या', 'हैं', '?', 'आप', 'क्या', 'सलाह', 'देते', 'हैं', '?', 'आप', 'क्या', 'बेच', 'रहे', 'हैं', '?', 'आप', 'लोग', 'कब', 'खुले', 'हैं', '?', 'आपके', 'घंटे', 'क्या', 'हैं', 'ऑपरेशन', 'के', 'घंटे', 'मेरे', 'पास', 'कौन', 'से', 'स्टॉक', 'हैं', '?', 'मेरे', 'शेयर', 'कैसे', 'हैं', '?', 'मैंने', 'किन', 'कंपनियों', 'में', 'निवेश', 'किया', 'है', '?', 'मैं', 'स्ट

### *Lemmatizing*

In [9]:
# TODO : figure our word lemmarization for hindi 

# lemmatizing our words
lemmatizer = WordNetLemmatizer()
  
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]

# removing duplicates
words = sorted(set(words))
classes = sorted(set(classes))

In [10]:
print(words)

['अपना', 'अरे', 'अलविदा', 'आप', 'आपका', 'आपकी', 'आपके', 'आपको', 'उत्पाद', 'उम्र', 'ऑपरेशन', 'कंपनियों', 'कब', 'कर', 'कल', 'का', 'कि', 'कितने', 'किन', 'किया', 'कुछ', 'के', 'कैसे', 'कौन', 'क्या', 'खरीदना', 'खुले', 'घंटे', 'चल', 'चाहता', 'जा', 'तुम्हारी', 'दिन', 'दिवस', 'देते', 'नमस्ते', 'नाम', 'निर्माता', 'निवेश', 'पास', 'प्रभात', 'फ्लोरियन', 'बता', 'बाद', 'बुलाऊं', 'बेच', 'मिलता', 'मिलते', 'मिलेंगे', 'मुझे', 'में', 'मेरे', 'मैं', 'मैंने', 'रहा', 'रहे', 'लोग', 'शुभ', 'शेयर', 'सकते', 'सलाह', 'साल', 'से', 'स्टॉक', 'हम', 'हूँ', 'हूं', 'है', 'हैं', 'हो']


### *Saving Data*

In [11]:
# adding all info to file
pickle.dump(words, open("words.pkl", 'wb'))
pickle.dump(classes, open("classes.pkl", 'wb'))
pickle.dump(docs, open("docs.pkl", 'wb'))

### *Creattig Training Data*

In [12]:
# representing words as numeric data using bag of words
# so that we could use this in a neural network
training = []
output_empty = [0]*len(classes)

for doc in docs:
  bag = []
  word_patterns = doc[0]
  word_patterns = [lemmatizer.lemmatize(word) for word in word_patterns]

  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

  # print(output_row)
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1
  training.append([bag, output_row])

### *Pre-Processing*

In [13]:
# shufling our data
random.shuffle(training)
training = np.array(training)

print(training.shape)


# spliting to X and y values
train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

# print(np.asarray(train_x))
print(train_y.shape)

(32, 2)
(32, 7)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


### *Building Neural Net*

In [14]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]), ), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))

In [15]:
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.09, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               9088      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 455       
Total params: 17,799
Trainable params: 17,799
Non-trainable params: 0
_________________________________________________________________


In [16]:
history = model.fit(train_x, train_y, epochs=200, batch_size=4, verbose=1)

Epoch 1/200
8/8 [==============================] - 1s 2ms/step - loss: 1.9724 - accuracy: 0.0625
Epoch 2/200
8/8 [==============================] - 0s 3ms/step - loss: 2.0321 - accuracy: 0.0938
Epoch 3/200
8/8 [==============================] - 0s 3ms/step - loss: 2.0473 - accuracy: 0.0625
Epoch 4/200
8/8 [==============================] - 0s 3ms/step - loss: 1.9493 - accuracy: 0.1562
Epoch 5/200
8/8 [==============================] - 0s 3ms/step - loss: 1.9722 - accuracy: 0.0938
Epoch 6/200
8/8 [==============================] - 0s 2ms/step - loss: 1.9196 - accuracy: 0.1875
Epoch 7/200
8/8 [==============================] - 0s 2ms/step - loss: 1.9450 - accuracy: 0.1250
Epoch 8/200
8/8 [==============================] - 0s 3ms/step - loss: 1.9199 - accuracy: 0.1875
Epoch 9/200
8/8 [==============================] - 0s 2ms/step - loss: 1.9230 - accuracy: 0.1562
Epoch 10/200
8/8 [==============================] - 0s 3ms/step - loss: 1.9242 - accuracy: 0.1562
Epoch 11/200
8/8 [===========

In [17]:
model.save("ai_chatbot.h5", history)

## EXECUTING CHATBOT

In [18]:
# imports
import random
import json
import pickle   # for serialization
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model

# lemmatizing our words
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### *Loading Our Intents*

In [19]:
# loading file
from google.colab import files
uploaded = files.upload()

Saving intents_hin.json to intents_hin (1).json


In [20]:
intents =  json.loads(uploaded['intents_hin.json'])
print(intents)
print(intents['उद्देश्य'][2])
print(intents['उद्देश्य'][2]['उपनाम'])

# intents -> उद्देश्य
# tag -> उपनाम
# patterns -> प्रतिरूप
# responces -> प्रतिक्रियाओं

{'उद्देश्य': [{'उपनाम': 'अभिवादन', 'प्रतिरूप': ['नमस्ते', 'अरे', 'शुभ दिवस', 'शुभ प्रभात', 'क्या चल रहा है'], 'प्रतिक्रियाओं': ['नमस्ते', 'मेरे द्वारा आपके लिए क्या किया जा सकता है?']}, {'उपनाम': 'अलविदा', 'प्रतिरूप': ['बाद में मिलते है', 'मैं जा रहा हूँ', 'अलविदा', 'आपका दिन शुभ हो', 'हम बाद में मिलेंगे', 'आप से कल मिलता हूं'], 'प्रतिक्रियाओं': ['अलविदा', 'तुमसे मिलकर खुशी हुई']}, {'उपनाम': 'उम्र', 'प्रतिरूप': ['आपका निर्माता कितने साल का है', 'फ्लोरियन कि उम्र क्या है?', 'तुम्हारी उम्र क्या हैं', 'आपकी उम्र क्या है?', 'उम्र?'], 'प्रतिक्रियाओं': ['मेरा निर्माता २१ साल का है!', '२१ साल!', '२१']}, {'उपनाम': 'नाम', 'प्रतिरूप': ['आपका नाम क्या है', 'मैं आपको क्या बुलाऊं?', 'आपका नाम क्या है', 'आप कौन हैं', 'क्या आप मुझे अपना नाम बता सकते हैं'], 'प्रतिक्रियाओं': ['आप मुझे नियुरल कह सकते हैं!', 'मैं नियुरल हूं', 'मैं नियुरल हूं, फ्लोरियन का सहायक']}, {'उपनाम': 'दुकान', 'प्रतिरूप': ['मैं कुछ खरीदना चाहता हूं', 'आपके उत्पाद क्या हैं?', 'आप क्या सलाह देते हैं?', 'आप क्या बेच रहे हैं?'], 'प्रति

### *Loading up our saved files*

In [21]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
docs = pickle.load(open('docs.pkl', 'rb'))

print(words)
print(classes)
print(docs)

model = load_model("ai_chatbot.h5")

['अपना', 'अरे', 'अलविदा', 'आप', 'आपका', 'आपकी', 'आपके', 'आपको', 'उत्पाद', 'उम्र', 'ऑपरेशन', 'कंपनियों', 'कब', 'कर', 'कल', 'का', 'कि', 'कितने', 'किन', 'किया', 'कुछ', 'के', 'कैसे', 'कौन', 'क्या', 'खरीदना', 'खुले', 'घंटे', 'चल', 'चाहता', 'जा', 'तुम्हारी', 'दिन', 'दिवस', 'देते', 'नमस्ते', 'नाम', 'निर्माता', 'निवेश', 'पास', 'प्रभात', 'फ्लोरियन', 'बता', 'बाद', 'बुलाऊं', 'बेच', 'मिलता', 'मिलते', 'मिलेंगे', 'मुझे', 'में', 'मेरे', 'मैं', 'मैंने', 'रहा', 'रहे', 'लोग', 'शुभ', 'शेयर', 'सकते', 'सलाह', 'साल', 'से', 'स्टॉक', 'हम', 'हूँ', 'हूं', 'है', 'हैं', 'हो']
['अभिवादन', 'अलविदा', 'उम्र', 'घंटे', 'दुकान', 'नाम', 'शेयर']
[(['नमस्ते'], 'अभिवादन'), (['अरे'], 'अभिवादन'), (['शुभ', 'दिवस'], 'अभिवादन'), (['शुभ', 'प्रभात'], 'अभिवादन'), (['क्या', 'चल', 'रहा', 'है'], 'अभिवादन'), (['बाद', 'में', 'मिलते', 'है'], 'अलविदा'), (['मैं', 'जा', 'रहा', 'हूँ'], 'अलविदा'), (['अलविदा'], 'अलविदा'), (['आपका', 'दिन', 'शुभ', 'हो'], 'अलविदा'), (['हम', 'बाद', 'में', 'मिलेंगे'], 'अलविदा'), (['आप', 'से', 'कल', 'मिलता', 'हूं'],

### *Creating Helper Functions*



In [22]:
def clean_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words = clean_sentence(sentence)
  bag = [0] * len(words)

  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1
  
  return np.array(bag)

def predict_class(sentence):
  ERROR_THRESHOLD = 0.25

  bow = bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]

  results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

  results.sort(key=lambda x: x[1], reverse=True)

  return_list = []
  for r in results:
    return_list.append({ 'intent': classes[r[0]], 'probability': str(r[1]) })

  return return_list

def get_response(intents_list, intents_json):
  # print(intents_list)

  tag = intents_list[0]['intent']
  list_of_intents = intents_json['उद्देश्य']
  for i in list_of_intents:
    if i['उपनाम'] == tag:
      result = random.choice(i['प्रतिक्रियाओं'])
      break
  return result


## Running BOT

In [ ]:
while True:
  message = input('')
  ints = predict_class(message)
  # print("class : ", ints)
  res = get_response(ints, intents)
  print(res, "\n")

नमस्ते
मेरे द्वारा आपके लिए क्या किया जा सकता है? 

तुम्हारी उम्र क्या हैं
मेरा निर्माता २१ साल का है! 

मेरे पास कौन सी कंपनियां हैं
आपके पास निम्नलिखित शेयर हैं: ABBV, APPL, FB, NVDA और S&P 500 इंडेक्स का एक ETF! 

